In [ ]:
# Grafo
edges = {(0, 1): 5, (0, 2): 8, (1, 2): 2, (1, 3): 7, (2, 3): 4}
num_nodes = 4

# Hp = 11*Z0 - 17.5*Z1 - 28*Z2 - 17*Z3 + 11.5*Z4 +
#      13.5*(-Z0*Z2 + Z1*Z2 + Z2*Z3 - Z2*Z4 + Z0*Z1 - Z0*Z3 - Z1*Z4)

# Coeficientes lineales de Hp (11 * Z0 - 17.5 * Z1...)
linear_coefs = [11, -17.5, -28, -17, 11.5]

# Coeficientes cuadráticos Ej: {(2, 3): 13.5} -> 13.5 * Z2 * Z3
quadra_coefs = {(0, 1): 13.5, (0, 2): -13.5, (0, 3): -13.5, (1, 2): 13.5,
                (1, 4): -13.5, (2, 3): 13.5, (2, 4): -13.5}

# edges = {(0, 1): q_0,
#          (0, 2): q_1,
#          (1, 2): q_2,
#          (1, 3): q_3,
#          (2, 3): q_4}

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Imprime un grafo dirigido pesado
# Entrada:
#     graph_nodes: Lista de nodos del grafo
#     graph_edges: Diccionario {vértice: peso} donde vértice es (n1, n2)
#     layout:      Para especificar la posición de los nodos
def print_graph(graph_nodes, graph_edges, layout=None):
    G = nx.DiGraph()

    # Nodes
    G.add_nodes_from(graph_nodes)

    # Edges
    for pair, weight in graph_edges.items():
        G.add_edge(pair[0], pair[1], weight=weight)

    # Print graph
    if layout is None:
        layout = nx.spring_layout(G)

    plt.figure(3, figsize=(6, 2))
    nx.draw(G, layout, with_labels=True, node_size=600)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)
    #plt.savefig("../../latex/img/primer_grafo/primer_grafo.png")

print_graph(range(num_nodes), edges, layout=print_graph_layout)

In [ ]:
# Función de coste binaria con valores {0, 1}
cost_function = "\
5*x_01 + 8*x_02 + 2*x_12 + 7*x_13 + 4*x_23 + \
27*(x_01 + x_02 - 1)**2 + \
27*(x_01 - x_12 - x_13)**2 + \
27*(x_02 + x_12 - x_23)**2"

# Evalúa una solución concreta según la función de coste para la versión QUBO
# Entrada:
#     - bits: Cadena con los valores de los qubits medidos
#             Orden: q_4 q_3 q_2 q_1 q_0
def eval_cost_function(bits):
    assert len(edges) == len(bits), "Error in cost_function: Length of bits"

    inv_bits = bits[::-1]  # Los qubits están en orden inverso

    param_dict = {}
    # Asignación entre x_ij -> q_n
    # Establecido por el orden de edges. Ej: x_12 -> q_2 (esto es, qubit nº2)
    for n, (i, j) in enumerate(edges):
        param_dict[f"x_{i}{j}"] = int(inv_bits[n])

    return eval(cost_function, param_dict)

In [ ]:
# Show min of the cost function
import itertools

results = {}
for z in itertools.product(["0", "1"], repeat=len(edges)):
    z = ''.join(z)
    results[z] = eval_cost_function(z)

results = {k: v for k, v in sorted(results.items(), key=lambda x: x[1])}
for k in results:
    print(f"{k}: {results[k]}")

In [ ]:
from qiskit import QuantumCircuit

def generate_qaoa_circuit(theta):
    assert len(theta) % 2 == 0, "Error in parameters (Beta, Gamma)"

    nqubits = len(edges)  # Tantos qubits como aristas tenga el grafo
    circuit = QuantumCircuit(nqubits)

    layers = int(len(theta) / 2)
    beta = theta[:layers]
    gamma = theta[layers:]

    # |v0>
    for i in range(nqubits):
        circuit.h(i)

    for p in range(layers):
        circuit.barrier()

        # Hp
        for q_idx, coef in enumerate(linear_coefs):
            circuit.rz(coef * 2 * gamma[p], q_idx)
        for q_idxs, coef in quadra_coefs.items():
            circuit.rzz(coef * 2 * gamma[p], q_idxs[0], q_idxs[1])

        circuit.barrier()

        # Hm
        for q_idx in range(nqubits):
            circuit.rx(beta[p] * 2, q_idx)

    circuit.measure_all()
    return circuit

In [ ]:
def compute_expectation(counts):
    media = 0
    len_count = 0
    for bits, count in counts.items():
        cost = eval_cost_function(bits)
        media += cost * count
        len_count += count

    return media/len_count

In [ ]:
# Simulador
from qiskit import Aer
from scipy.optimize import minimize

backend = Aer.get_backend('aer_simulator')
shots = 1024

def execute_circuit(theta):
    qc = generate_qaoa_circuit(theta)
    counts = backend.run(qc, shots=shots).result().get_counts()
    return compute_expectation(counts)

num_layers = 1
theta_res = minimize(execute_circuit, [1.0, 1.0] * num_layers, method = "COBYLA")
theta_res

In [ ]:
from qiskit.visualization import plot_histogram
backend = Aer.get_backend('aer_simulator')
shots = 1024

qc = generate_qaoa_circuit(theta_res.x)
counts = backend.run(qc, shots=shots).result().get_counts()
#plot_histogram(normalized_counts, figsize=(13, 6.83),
#               filename="../../latex/resultados/img/primer_grafo/sin_restriccion_extra/primer_paper_aer_resultado.png")
normalized_counts = {key: val / shots for (key, val) in counts.items()}
plot_histogram(normalized_counts, figsize=(13, 6.83))

In [ ]:
# Gamma function
import numpy as np
import matplotlib.pyplot as plt

def gamma_function():
    x = np.linspace(0.3, 1.5, 120)
    y = []
    beta = 1.0
    for gamma in x:
        y.append(execute_circuit([beta, gamma]))

    plt.plot(x, y)
    # plt.savefig("../../latex/resultados/img/primer_grafo/sin_restriccion_extra/primer_paper_p_27_gamma_fun.png")
    plt.show()

gamma_function()

In [ ]:
# 3D Gamma function
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import itertools

def gamma_function_3d():
    beta = np.linspace(0.3, 1.5, 120)
    gamma = np.linspace(0.3, 1.5, 120)
    z = np.zeros([len(gamma), len(beta)])
    beta, gamma = np.meshgrid(beta, gamma)

    for i in range(z.shape[0]):
        for j in range(z.shape[1]):
            z[i][j] = execute_circuit([beta[i][j], gamma[i][j]])

    fig = plt.figure()
    axis = plt.axes(projection='3d')
    figure = axis.plot_surface(gamma, beta, z, cmap=cm.coolwarm, linewidth=0, antialiased=False)
    fig.colorbar(figure, shrink=0.5, aspect=5)
    # plt.savefig("../../latex/resultados/img/primer_grafo/sin_restriccion_extra/primer_paper_p_27_gamma_fun.png")
    plt.show()

gamma_function_3d()

In [ ]:
# Statistics
def max_global_statistics(num_layers=1, num_generations=100):
    max_statistics = {}
    global_statistics = {}
    for iteration in range(0, num_generations):
        theta_res = minimize(execute_circuit, [1.0, 1.0] * num_layers, method = "COBYLA")
        qc = generate_qaoa_circuit(theta_res.x)
        counts = backend.run(qc, shots=shots).result().get_counts()

        # Max statistics
        max_path = max(counts, key=counts.get)
        if max_path not in max_statistics:
            max_statistics[max_path] = 0
        max_statistics[max_path] += 1

        # Global statistics
        for (path, num_appearances) in counts.items():
            if path not in global_statistics:
                global_statistics[path] = 0
            global_statistics[path] += num_appearances

    for path in global_statistics:
        global_statistics[path] = global_statistics[path] / shots / num_generations  # Normalize

    for path in max_statistics:
        max_statistics[path] = max_statistics[path] / num_generations  # Normalize

    max_statistics = sorted(max_statistics.items(), key=lambda x: x[1], reverse=True)
    global_statistics = sorted(global_statistics.items(), key=lambda x: x[1], reverse=True)
    return max_statistics, global_statistics


interval_num_layers = (4, 5)
for p in range(interval_num_layers[0], interval_num_layers[1] + 1):
    max_st, global_st = max_global_statistics(num_layers=p, num_generations=100)
    print(f"Max statistics (p = {p}): ", str(max_st))
    print(f"Global statistics (p = {p}): ", str(global_st))
    print()

In [ ]:
# Max statistics (p = 1):  [('00000', 0.747), ('10001', 0.121), ('00010', 0.054), ('01001', 0.038), ('10000', 0.01), ('10010', 0.008), ('11011', 0.006), ('10101', 0.005), ('01011', 0.003), ('00001', 0.003), ('01000', 0.002), ('10011', 0.001), ('11001', 0.001), ('11000', 0.001)]
# Global statistics (p = 1):  [('00000', 0.2606376953125), ('00010', 0.14180078125), ('00001', 0.1273720703125), ('00011', 0.0846796875), ('10001', 0.064578125), ('10101', 0.0404716796875), ('01000', 0.0364150390625), ('00100', 0.0328359375), ('00110', 0.0301103515625), ('10000', 0.0295146484375), ('01001', 0.0180283203125), ('00101', 0.0179658203125), ('10010', 0.012953125), ('00111', 0.0123505859375), ('01010', 0.012310546875), ('11000', 0.008453125), ('01011', 0.0076533203125), ('01100', 0.0066494140625), ('10011', 0.0064658203125), ('10100', 0.0063330078125), ('11001', 0.00612109375), ('11101', 0.0058369140625), ('10111', 0.005314453125), ('01101', 0.0049912109375), ('01110', 0.0043125), ('11011', 0.003833984375), ('10110', 0.00343359375), ('11010', 0.00278515625), ('01111', 0.002380859375), ('11100', 0.001890625), ('11111', 0.0010205078125), ('11110', 0.0005)]

# Max statistics (p = 2):  [('01001', 0.381), ('10101', 0.092), ('10010', 0.087), ('00000', 0.071), ('11010', 0.065), ('10000', 0.056), ('00001', 0.046), ('10001', 0.035), ('11001', 0.03), ('00010', 0.024), ('00101', 0.017), ('11011', 0.017), ('01000', 0.016), ('01101', 0.014), ('11101', 0.012), ('10111', 0.006), ('01010', 0.006), ('10110', 0.005), ('10011', 0.005), ('00011', 0.004), ('10100', 0.003), ('11000', 0.003), ('01011', 0.002), ('00100', 0.002), ('11110', 0.001)]
# Global statistics (p = 2):  [('01001', 0.1703583984375), ('00001', 0.0641953125), ('00000', 0.0621162109375), ('10101', 0.058279296875), ('11001', 0.0532158203125), ('10010', 0.05028515625), ('10000', 0.0404052734375), ('10001', 0.0390205078125), ('01011', 0.037263671875), ('01000', 0.0371845703125), ('00010', 0.033548828125), ('11010', 0.0316279296875), ('00101', 0.028376953125), ('01101', 0.0270654296875), ('11101', 0.02637890625), ('11011', 0.023724609375), ('10100', 0.022974609375), ('10011', 0.02228515625), ('01010', 0.021111328125), ('11000', 0.019705078125), ('00011', 0.01921484375), ('00100', 0.019060546875), ('10111', 0.0172333984375), ('10110', 0.016107421875), ('01111', 0.01019140625), ('11111', 0.008515625), ('00111', 0.0079072265625), ('11100', 0.007673828125), ('01100', 0.0075322265625), ('00110', 0.0074814453125), ('11110', 0.0050400390625), ('01110', 0.0049189453125)]

# Max statistics (p = 3):  [('10101', 0.548), ('00001', 0.187), ('00000', 0.117), ('11010', 0.055), ('11101', 0.036), ('11001', 0.022), ('10110', 0.007), ('00010', 0.007), ('10010', 0.004), ('01101', 0.004), ('01001', 0.003), ('00101', 0.003), ('11011', 0.001), ('10000', 0.001), ('00100', 0.001), ('01000', 0.001), ('01011', 0.001), ('10100', 0.001), ('01010', 0.001)]
# Global statistics (p = 3):  [('10101', 0.1298017578125), ('00000', 0.07260546875), ('00001', 0.0687490234375), ('10010', 0.0623427734375), ('01101', 0.0592236328125), ('11101', 0.04654296875), ('11010', 0.044693359375), ('01001', 0.042392578125), ('10100', 0.03910546875), ('10001', 0.034166015625), ('10000', 0.0335458984375), ('10111', 0.0329287109375), ('11000', 0.030796875), ('01000', 0.029337890625), ('00100', 0.0247646484375), ('11001', 0.02325), ('00010', 0.0220693359375), ('00101', 0.0205537109375), ('10110', 0.0202529296875), ('01011', 0.0193388671875), ('00011', 0.017576171875), ('11111', 0.016662109375), ('11100', 0.0159130859375), ('01010', 0.014), ('01100', 0.0133046875), ('10011', 0.01310546875), ('11110', 0.0130478515625), ('11011', 0.009587890625), ('01111', 0.009138671875), ('00110', 0.00854296875), ('01110', 0.006533203125), ('00111', 0.0061259765625)]

# Max statistics (p = 4):  [('10010', 0.17), ('01001', 0.153), ('00000', 0.144), ('10101', 0.111), ('00001', 0.066), ('10000', 0.05), ('10100', 0.042), ('01010', 0.04), ('01000', 0.036), ('00010', 0.027), ('11101', 0.024), ('00101', 0.022), ('11001', 0.019), ('11010', 0.019), ('11011', 0.015), ('10001', 0.012), ('10111', 0.011), ('10011', 0.011), ('01101', 0.008), ('01011', 0.007), ('10110', 0.005), ('00100', 0.004), ('11000', 0.002), ('00011', 0.002)]
# Global statistics (p = 4):  [('01001', 0.0788369140625), ('10010', 0.0786611328125), ('10101', 0.070044921875), ('00000', 0.0672861328125), ('00001', 0.051326171875), ('10000', 0.04697265625), ('00101', 0.0406728515625), ('10100', 0.03939453125), ('01000', 0.0382978515625), ('00010', 0.0353203125), ('11101', 0.0344697265625), ('10001', 0.0335712890625), ('01010', 0.0335712890625), ('11001', 0.033396484375), ('01101', 0.02933984375), ('11011', 0.027583984375), ('11010', 0.026521484375), ('10111', 0.025197265625), ('10110', 0.0229921875), ('01011', 0.022595703125), ('00011', 0.02115625), ('10011', 0.020666015625), ('00100', 0.0204560546875), ('11000', 0.018044921875), ('11111', 0.014021484375), ('11100', 0.0111767578125), ('00111', 0.01075), ('00110', 0.01062109375), ('11110', 0.0104404296875), ('01111', 0.0096396484375), ('01100', 0.0096171875), ('01110', 0.007357421875)]

# Max statistics (p = 5):  [('10000', 0.208), ('01001', 0.203), ('00101', 0.136), ('00001', 0.135), ('10010', 0.075), ('00010', 0.074), ('10101', 0.06), ('10100', 0.058), ('11011', 0.011), ('10001', 0.011), ('00000', 0.007), ('11001', 0.006), ('11101', 0.004), ('10110', 0.004), ('11010', 0.003), ('01011', 0.001), ('01101', 0.001), ('01010', 0.001), ('10111', 0.001), ('00100', 0.001)]
# Global statistics (p = 5):  [('00001', 0.1075478515625), ('01001', 0.101423828125), ('00101', 0.085876953125), ('10101', 0.057087890625), ('10100', 0.0532041015625), ('00010', 0.0529443359375), ('10000', 0.0518603515625), ('10010', 0.050685546875), ('11001', 0.0419404296875), ('11011', 0.0388642578125), ('10001', 0.0336865234375), ('00000', 0.028517578125), ('11101', 0.02641796875), ('01011', 0.02351171875), ('11010', 0.0232548828125), ('10110', 0.020583984375), ('01000', 0.0200751953125), ('00100', 0.01990234375), ('10111', 0.0195654296875), ('01101', 0.0187412109375), ('11100', 0.0171962890625), ('11000', 0.0151875), ('01010', 0.0150224609375), ('00011', 0.0118291015625), ('01111', 0.011712890625), ('10011', 0.0113076171875), ('00111', 0.011126953125), ('01110', 0.007884765625), ('01100', 0.0066611328125), ('11111', 0.0058154296875), ('00110', 0.0057666015625), ('11110', 0.004796875)]


In [ ]:
!dunstify -t $((30* 1000*60)) --urgency=critical "Ejecución completada"